### Mie First
Mie theory provides a route to the exact solution to Maxwell's equations for light interacting with a spherical object; this is done by expanding the incident and scattered
light waves in a basis of multi-polar functions; hence one usually speaks about the dipolar contribution to scattering, the quadrupolar contribution, the octupolar, and so on.  For very large particles, many orders of this expansion may be needed, but for small particles the scattering and absorption is dominated by the dipolar contribution.  Hence, for spherical nanoparticles that are less than or equal to about 10 nm in diameter, the Mie theory expansion is truncated after only the first term.  As a concrete example, the Mie theory expression for the absorption of a small spherical nanoparticle is as follows:

\begin{equation}
\sigma_{abs}(\lambda) = \frac{2 \pi}{\lambda} {\rm Im}\left( \alpha(\lambda) \right),
\end{equation}
where $\alpha(\lambda)$ is the dipole polarizability given by
\begin{equation}
\alpha(\lambda) = 4 \pi r^3 \left( \frac{\epsilon_p(\lambda) - \epsilon_s}{\epsilon_p(\lambda) + 2\epsilon_s} \right)
\end{equation}
where $r$ is the radius of the particle, $\epsilon_p(\lambda)$ denotes the wavelength-dependent dielectric function of the material the particle is made from, $\epsilon_s$ denotes the dielectric function of the surroundings (or solvent) that the particle is immersed in, and ${\rm Im}$ means we will only take the imaginary part of the quotient (the dielectric function of the particle will typically be complex; if it is not, there will be no absorption).

We can use WPTherml to get the refractive index of materials as a function of wavelength, which is equivalent to the dielectric function as a function of wavelength by
\begin{equation}
\epsilon_p(\lambda) = n_p(\lambda)^2,
\end{equation}
where $n_p(\lambda)$ is the refractive index of the particle. 


In [1]:
from wptherml.wpml import multilayer
from matplotlib import pyplot as plt
from wptherml.datalib import datalib
import numpy as np
from numpy import linalg as LA
import math

structure = {

        'Material_List': ['Air', 'Ag', 'Air'],
        'Thickness_List': [0,  200e-9, 0],
        'Lambda_List': [200e-9, 700e-9, 2000],
     
        }

### create the silver object - call it sphere for no particular reason
sphere = multilayer(structure)
### get the refractive index of Ag (since it is layer 1 based on the structure above)
n_Ag = sphere.layer_ri(1)
### get the dielectric function of Ag
eps_Ag = n_Ag * n_Ag

'''Lets compute the Mie theory absorption of a 3 nm Ag particle in water, which 
   has refractive index = 1.33, here!'''
r = 3e-9 ### fill in radius here, use meters!
eps_s =  1.00 ### fill in dielectric function of water here!
quotient = (eps_Ag - eps_s)/(eps_Ag + 2*eps_s) ### compute the quotient whose Imaginary part you need from Mie theory expression here
alpha = 4*np.pi*r**3 * quotient
pre = 2*np.pi/sphere.lambda_array ### compute pre-factor that depends on wavelength and radius here!
sigma_abs = pre*np.imag(alpha) ### compute full absorption spectrum here!

plt.plot(1240/(sphere.lambda_array*1e9), sigma_abs, 'red')
plt.xlim(1.5, 4)
plt.show()

lidx = np.argmax(sigma_abs)
print(lidx)
print(sigma_abs[lidx])
print(sphere.lambda_array[lidx])

ModuleNotFoundError: No module named 'wptherml'

In [1]:

def Euler (HA0, muA, VintA, gammaA, DA, h, t, tau):  
    ###Defined Hamiltonian at the current time    
    HA = HA0- EField(t, tau) * muA 
    ### Define the time derivarive of our density matrix at the current time
    ### by evaluating the Liouville-Lindblad equation
    DAdot = Liouville(HA,DA) + Lindblad(DA, gammaA)
    ###
    DAnew = DA + h*DAdot
    return DAnew

def Lindblad(DA, gammaA):
    dim = len(DA)
    LDA = np.zeros_like(DA)
    ### need |g><g|
    bra_1 = CreateBas(dim, 0)
    gm = Form_Rho(bra_1)
    
    for k in range(1,dim):
        bra_k = CreateBas(dim, k)
        km = Form_Rho(bra_k)
        
        ### first term 2*gam*<k|D|k>|g><g|
        t1 = 2*gammaA*DA[k][k]*gm
        ### second term is |k><k|*D
        t2 = np.dot(km,DA)
        ### third term is  D*|k><k|
        t3 = np.dot(DA, km)
        LDA = LDA + t1 - gammaA*t2 - gammaA*t3
        
    return LDA
       
### Take commutator of H and D to give Ddot
def Liouville(HA, DA):
    ci = 0.+1j
    return -ci*(np.dot(HA,DA) - np.dot(DA, HA))

def EField(t, tau):
    Ef = 0
    if t<tau:
        Ef = 0.001*np.sin(t*np.pi/tau)*np.sin(t*np.pi/tau)*np.sin(0.1192*t)
    return Ef

def Form_Rho(Psi):

    DA = np.outer(Psi,np.conj(Psi))
    return DA

### Creates basis vector for state k
### k=0 -> ground state, k=1 -> first excited-state, etc
def CreateBas(dim, k):
    bas = np.zeros(dim)
    bas[k] = 1
    return bas

def Euler (HB0, muB, VintB, gammaB, DB, h, t, tau):  
    ###Defined Hamiltonian at the current time    
    HB = HB0- EField(t, tau) * muB 
    ### Define the time derivarive of our density matrix at the current time
    ### by evaluating the Liouville-Lindblad equation
    DBdot = Liouville(HB,DB) + Lindblad(DB, gammaB)
    ###
    DBnew = DB + h*DBdot
    return DBnew

def Lindblad(DB, gammaB):
    dim = len(DB)
    LDB = np.zeros_like(DB)
    ### need |g><g|
    bra_1 = CreateBas(dim, 0)
    gm = Form_Rho(bra_1)
    
    for k in range(1,dim):
        bra_k = CreateBas(dim, k)
        km = Form_Rho(bra_k)
        
        ### first term 2*gam*<k|D|k>|g><g|
        t1 = 2*gammaB*DB[k][k]*gm
        ### second term is |k><k|*D
        t2 = np.dot(km,DB)
        ### third term is  D*|k><k|
        t3 = np.dot(DB, km)
        LDB = LDB + t1 - gammaB*t2 - gammaB*t3
        
    return LDB
       
### Take commutator of H and D to give Ddot
def Liouville(HB, DB):
    ci = 0.+1j
    return -ci*(np.dot(HB,DB) - np.dot(DB, HB))

def EField(t, tau):
    Ef = 0
    if t<tau:
        Ef = 0.001*np.sin(t*np.pi/tau)*np.sin(t*np.pi/tau)*np.sin(0.1192*t)
    return Ef

def Form_Rho(Psi):

    DB = np.outer(Psi,np.conj(Psi))
    return DB

### Creates basis vector for state k
### k=0 -> ground state, k=1 -> first excited-state, etc
def CreateBas(dim, k):
    bas = np.zeros(dim)
    bas[k] = 1
    return bas

### Vint will take three vectors: 
### R the separation vector, 
### MUA is the vector representing the
### transition dipole moment on one of your particles (does not
### change with time) and MUB is the vector
### representing the instantaneous dipole moment on the other particle
### (it does change with time)
def Vint(R, MUA, MUB):
    n = 1
    normr = math.sqrt(np.dot(R,R))
    oer2 = 1./(normr**2)
    pre = 1./(n**2 * normr**3)
    
    t1 = np.dot(MUA,MUB)
    t2 = np.dot(MUA,R)
    t3 = np.dot(R,MUB)
    
    Vint_val = pre*(t1 - 3*oer2*t2*t3)
    
    return Vint_val

A slightly more sophisticated update scheme is given by the family of methods known as 
[Runge-Kutta methods](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods), and a particular implementation of this method can be found below in the function called *RK4*.



In [ ]:
def RK4(HA0, muA, VintA, gammaA, DA, h, t, tau):
    k1 = np.zeros_like(DA)
    k2 = np.zeros_like(DA)
    k3 = np.zeros_like(DA)
    k4 = np.zeros_like(DA)
    DA1 = np.zeros_like(DA)
    DA2 = np.zeros_like(DA)
    DA3 = np.zeros_like(DA)
    DA4 = np.zeros_like(DA)
    DAf = np.zeros_like(DA)
    
    ### Get k1
    HA1 = HA0 - EField(t, tau)*muA + VintA
    DA1 = DA    
    k1 = h*Liouville(HA1,DA1) + h*Lindblad(DA1, gammaA)
    
    ## Update H and D and get k2
    HA2 = HA0 - EField(t+h/2, tau)*muA 
    DA2 = DA+k1/2.
    k2 = h*Liouville(HA2, DA2) + h*Lindblad(DA2, gammaA)
    
    ### UPdate H and D and get k3
    HA3 = HA2
    DA3 = DA+k2/2
    k3 = h*Liouville(HA3, DA3) + h*Lindblad(DA3, gammaA) 
    
    ### Update H and D and get K4
    HA4 = HA0 - EField(t+h, tau)*muA + VintA
    DA4 = DA+k3
    k4 = h*Liouville(HA4, DA4) + h*Lindblad(DA4, gammaA)
    
    DAf = DA + (1/6.)*(k1 + 2.*k2 + 2*k3 + k4)
    return DAf
''' A second RK4 function is not needed, the original RK4 function
    will work for DA and DB 
def RK4(HB0, muB, VintB, gammaB, DB, h, t, tau):
    k1 = np.zeros_like(DB)
    k2 = np.zeros_like(DB)
    k3 = np.zeros_like(DB)
    k4 = np.zeros_like(DB)
    DB1 = np.zeros_like(DB)
    DB2 = np.zeros_like(DB)
    DB3 = np.zeros_like(DB)
    DB4 = np.zeros_like(DB)
    DBf = np.zeros_like(DB)
    
    ### Get k1
    HB1 = HB0 - EField(t, tau)*muB + VintB
    DB1 = DB    
    k1 = h*Liouville(HB1,DB1) + h*Lindblad(DB1, gammaB)
    
    ## Update H and D and get k2
    HB2 = HB0 - EField(t+h/2, tau)*muB 
    DB2 = DB+k1/2.
    k2 = h*Liouville(HB2, DB2) + h*Lindblad(DB2, gammaB)
    
    ### UPdate H and D and get k3
    HB3 = HB2
    DB3 = DB+k2/2
    k3 = h*Liouville(HB3, DB3) + h*Lindblad(DB3, gammaB) 
    
    ### Update H and D and get K4
    HB4 = HB0 - EField(t+h, tau)*muB + VintB
    DB4 = DB+k3
    k4 = h*Liouville(HB4, DB4) + h*Lindblad(DB4, gammaB)
    
    DBf = DB + (1/6.)*(k1 + 2.*k2 + 2*k3 + k4)
    return DBf
'''

We will compare the results of the two methods (RK4 and Euler) in 
reproducing the Mie theory spectrum for a 3 nm Ag atom.  
The following Hamiltonian parameters will be used, and should in principle provide good agreement provided the equations of motion are accurately solved:
In atomic units, $E_2 = 0.1275, \mu_{12} = 58.$, and $\gamma=0.0017$ gives excellent agreement with the Mie theory spectrum, as we will see below.

In [ ]:
### Set up some parameters for the quantum dynamics simulation
dt = 0.1
tau = 100 #150.
gammaA = 0.0017
gammaB = 0.0017
muA = 8.854e-12
muB = 8.854e-12
#mu_au_to_si = 8.47835326e-30
#E_au_to_si = 5.14220652e11
mu_zA = 58.
mu_zB = 58.
R = 3e-9

### Create some arrays
MUZA = np.zeros((2,2),dtype=complex)
VintA = np.zeros((2,2),dtype=complex)
### Density matrix for RK4 updates
DA_RK4 = np.zeros((2,2),dtype=complex)
### Density matrix for Euler updates
DA_EU = np.zeros((2,2),dtype=complex)
HA0 = np.zeros((2,2))

### initialize values of the arrays for Hamiltonian and Density matrices
HA0[0][0] = 0.1275
DA_RK4[0][0] = 1.+0j
DA_EU[0][0] = 1.+0j
MUZA[0][1] = mu_zA
MUZA[1][0] = mu_zA

### need vectors to pass to the function Vint...
### these vectors will store information about R, MUZA, MUZB (all static)
### and we will also create additional vectors that store the time-dependent 
### quantities
### arrays for static quantities first
### there are 3 components for the x-, y-, and z- components
### of these vector quantities
MUA_VEC = np.zeros(3)
MUB_VEC = np.zeros(3)
R_VEC = np.zeros(3)
### actually assign the values of the static quantities...
### note only the z-component (index 2) is non-zero for now!
MUA_VEC[2] = mu_zA
MUB_VEC[2] = mu_zA
R_VEC[2] = R
### time-dependent quantities - still x-, y-, and z- components
### in principle, though only the z-component (index 2) will be non-zero
MUA_OF_T = np.zeros(3)
MUB_OF_T = np.zeros(3)



### create arrays for time-dependent quantities
Nsteps = 40000
ez = np.zeros(Nsteps)
### array for mu(t) for RK4 updates
muA_of_t_rk4 = np.zeros(Nsteps,dtype=complex)
### array for mu(t) for Euler updates
muA_of_t_eu = np.zeros(Nsteps,dtype=complex)
time = np.zeros(Nsteps)
energy = np.zeros(Nsteps)

### Run the dynamics for both systems simultaneously
for i in range(0,Nsteps):
    ### reciprocal axis
    energy[i] = np.pi*2*(i+1)/(Nsteps*dt)
    ### time access
    time[i] = i*dt
    ### time-dependent electric field
    ez[i] = EField(i*dt, tau)
    ### update to the Density matrix using RK4
    DA_RK4 = RK4(HA0, MUZA, VintA, gammaA, DA_RK4, dt, dt*i, tau)
    ### use np.matmul to take the matrix product of DA_RK4 and MUZA
    DMA = np.matmul(DA_RK4, MUZA)
    ### DMA is now the matrix product of DA_RK4 and MUZA
    ### TO GET THE TRACE, WE JUST SUM THE DIAGONAL ELEMENTS...
    ### AND WE WANT TO STORE THE TRACE DIRECTLY IN OUR MUA_OF_T 
    ### ARRAY FOR USE IN THE Vint FUNCTION
    MUA_OF_T[2] = DMA[0][0]+DMA[1][1]
    ### Vint function currently returns a scalar value..
    ### these scalar values are the off-diagonal elements of the VintB/VintA 
    ### matrices we will pass to RK4
    VintB[0][1] = Vint(R, MUB_VEC, MUA_OF_T)
    VintB[1][0] = Vint(R, MUB_VEC, MUA_OF_T)

    #### NOW CALL TRY TO REPEAT THE PROCESS TO UPDATE DB_RK4 HERE!
    

print(energy[0])
plt.plot(time, muA_of_t_eu, 'red', time, muA_of_t_rk4, 'b--', time, ez, 'green')
plt.show()

### Create some arrays
MUZB= np.zeros((2,2),dtype=complex)
VintB = np.zeros((2,2),dtype=complex)
### Density matrix for RK4 updates
DB_RK4 = np.zeros((2,2),dtype=complex)
### Density matrix for Euler updates
DB_EU = np.zeros((2,2),dtype=complex)
HB0 = np.zeros((2,2))

### initialize values of the arrays for Hamiltonian and Density matrices
HB0[0][0] = 0.1275
DB_RK4[0][0] = 1.+0j
DB_EU[0][0] = 1.+0j
MUZB[0][1] = mu_zB
MUZB[1][0] = mu_zB

### create arrays for time-dependent quantities
Nsteps = 40000
ez = np.zeros(Nsteps)
### array for mu(t) for RK4 updates
muB_of_t_rk4 = np.zeros(Nsteps,dtype=complex)
### array for mu(t) for Euler updates
muB_of_t_eu = np.zeros(Nsteps,dtype=complex)
time = np.zeros(Nsteps)
energy = np.zeros(Nsteps)

### Run the dynamics
''' NOTE THIS SEPARATE LOOP OVER TIME IS NOT NEEDED! 
    YOU WANT TO UPDATE SYSTEM A AND SYSTEM B TOGETHER IN THE LOOP ABOVE!
    
for i in range(0,Nsteps):
    ### reciprocal axis
    energy[i] = np.pi*2*(i+1)/(Nsteps*dt)
    ### time access
    time[i] = i*dt
    ### time-dependent electric field
    ez[i] = EField(i*dt, tau)
    ### update to the Density matrix using RK4
    DB_RK4 = RK4(HB0, MUZB, VintB, gammaB, DB_RK4, dt, dt*i, tau)
    DMB = Tr(DB_RK4 * MUZB)
    VintB = Vint(R, DMA, MUZB)
    VintB = (1/R**3)(MUZB*DMA-((MUZB.R)(R*DMA)/R**2)
    DB_EU = Euler(HB0, MUZB, VintB, gammaB, DB_RK4, dt, dt*i, tau)
    ### Update to mu(t) using RK4
    DBMUB_RK4 = np.matmul(DB_RK4, MUZB)
    DBMUB_EU = np.matmul(DB_EU, MUZB)
    muB_of_t_rk4[i] = (DBMUB_RK4[0][0] + DBMUB_RK4[1][1])
    muB_of_t_eu[i] = (DBMUB_EU[0][0] + DBMUB_EU[1][1])
    ### add update using Euler step!!!
'''
print(energy[0])
plt.plot(time, muB_of_t_eu, 'black', time, muB_of_t_rk4, 'b--', time, ez, 'yellow')
plt.show()

'''
mu_freq_rk4 = np.fft.fft(mu_of_t_rk4)/(Nsteps)
ez_freq = np.fft.fft(ez)/(Nsteps)
alpha_rk4 = mu_freq_rk4/ez_freq
lam = 1e-9*1240/(energy*27.211) ### in nm

sigma_rk4 = 2*np.pi/(lam*eps0) * np.imag(alpha_rk4)
plt.plot(energy*27.211, sigma_rk4, 'red')
#plt.plot(1240/(sphere.lambda_array*1e9), sigma_abs, 'b--')
plt.xlim(1.5,4.0)
plt.ylim(0,1e-16)
#plt.plot(time, ez, 'red', time, mu_of_t, 'blue')
plt.show()
'''